# Objective

Establish the accuracy of a base LLM (Mistral) on dialogue summarization


# Setup

In [1]:
# @title Click run for setup

!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps -q "xformers<0.0.26" trl peft accelerate bitsandbytes

!pip install -q datasets evaluate bert-score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 35.6 MB/s eta 0:0

# Imports

In [2]:
import torch
import evaluate

from datasets import load_dataset
from tqdm import tqdm

from unsloth import FastLanguageModel

# Test Data

Source: https://huggingface.co/datasets/knkarthick/dialogsum

In [3]:
dataset = load_dataset("knkarthick/dialogsum")

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [4]:
test_data_size = 30

In [5]:
test_dataset = dataset['test'].shuffle(seed=42).select(range(test_data_size))

In [6]:
test_dialogues = [sample['dialogue'] for sample in test_dataset]
test_summaries = [sample['summary'] for sample in test_dataset]

# Model

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [8]:
FastLanguageModel.for_inference(model)

# Inference

In [9]:
summarization_prompt_template = """[INST]
Summarize the dialogue mentioned in the user input. Be specific and concise in your summary.
Ensure that you retain the entities mentioned in the dialogue in your summary.

### User Input:
{dialogue}
[/INST]
"""

In [10]:
predicted_summaries = []

In [11]:
for gold_dialogue in tqdm(test_dialogues):

    try:
        prompt = summarization_prompt_template.format(dialogue=gold_dialogue)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            use_cache=True,
            temperature=0,
            pad_token_id=tokenizer.eos_token_id
        )

        prediction = tokenizer.decode(
            outputs[0][inputs.input_ids.shape[-1]:],
            skip_special_tokens=True,
            cleanup_tokenization_spaces=True
        )

        predicted_summaries.append(prediction)

    except Exception as e:
        print(e) # log error and continue
        continue

100%|██████████| 30/30 [02:40<00:00,  5.35s/it]


# Evaluation

In [12]:
predicted_summaries[1]

'Person1 requests to send a package using first-class mail with insurance for $50. Person2 informs Person1 that they need to get stamps, including a book of 22 and three airmail, at the stamp window next to general delivery. Person1 inquires if they can also get money orders there, but is told they need to go to the window three windows down the hall for that.'

In [13]:
bert_scorer = evaluate.load("bertscore")

In [14]:
score = bert_scorer.compute(
    predictions=predicted_summaries,
    references=test_summaries,
    lang='en',
    rescale_with_baseline=True
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
sum(score['f1'])/len(score['f1'])

0.18470857044061026